In [15]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(['seaborn-ticks', 'seaborn-talk'])

import numpy as np
import pandas as pd
import xarray as xr

pd.set_option('precision', 2)
np.set_printoptions(precision=2, suppress=True, nanstr='nan', infstr='inf', threshold=5)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Notes

- **Theming**
    - Surprisingly enough, the embedded `reveal.js` will respect any custom styling set in **custom.css**, which means you can copy any theme from the [reveal repository](https://github.com/hakimel/reveal.js/tree/master/css/theme), drop it there, and it'll work out-of-the-box!
    - To apply a default theme, add the token "?theme={theme}" after the ".html" in your URL. Valid themes include "sky", "beige", "night", "serif".
- **Template**: https://github.com/datitran/jupyter-presentation-template
- **Slideshow Tutorial**: http://www.slideviper.oquanta.info/tutorial/slideshow_tutorial_slides.html#/2
- **Examples**:
    - https://github.com/doingmathwithpython/pycon-us-2016

// Styling
<style type="text/css">
.input_prompt, .input_area, .output_prompt {
display:none !important;
}
.reveal h1, .reveal h2 {
    font-family:"League Gothic"
}
</style>

# <center> Basic pandas and xarray </center>

<center>
    <b>Daniel Rothenberg</b> | darothen@mit.edu
    <p>January 19, 2017 - JP Student Luncheon</p>

---

<img src="figs/pandas_logo.png">

## What is [pandas](http://pandas.pydata.org)?

- Toolkit built on-top of NumPy to emulate R's `data.frame`
- Performance-focused
- Simple API with rich documentation
- Developed by quantitative finance folks ([@wesmckinn](http://www.twitter.com/wesmckinn))


### ... and why should I care?

- Make data processing and cleaning easy! 
- Make data analysis fast and fun!
- Open your data workflows to powerful new tools

### Use the right tool for your job!

<img src="figs/wrong_tool.jpg">

### How to follow along

1. Download this notebook from github, and run the code samples as we go.

2. Open an `ipython` interpreter and enter the code by hand.

<br>

``` shell
$ ipython
Python 3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:52:12)
Type "copyright", "credits" or "license" for more information.

IPython 5.1.0 -- An enhanced Interactive Python.
?         -> Introduction and overview of IPython's features.
%quickref -> Quick reference.
help      -> Python's own help system.
object?   -> Details about 'object', use 'object??' for extra details.

In [1]: <your code here>
```

## Basics and Overview

## [Series](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#series)

<blockquote>
    <b>A one-dimensional, labeled array capable of holding any data type.</b>
</blockquote>

<img src="figs/series.svg" style="width: 400px">

### Creating Series

From list-like data...

In [3]:
temperature_data = [21, 25, 16, 22, 16, 21, 15]
pd.Series(temperature_data)

0    21
1    25
2    16
3    22
4    16
5    21
6    15
dtype: int64

With custom index...

In [6]:
days_of_the_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday',
                    'Friday', 'Saturday', 'Sunday']
temperature_series = pd.Series(temperature_data, index=days_of_the_week)
temperature_series.head()

Monday       21
Tuesday      25
Wednesday    16
Thursday     22
Friday       16
dtype: int64

### Labeled Indexing

How would we find the temperature for Thursday?

In [8]:
for idx, val in enumerate(days_of_the_week):
    if val == "Thursday": break

idx, temperature_data[idx]

(3, 22)

... but what if we have labeled data?

In [9]:
temperature_series['Thursday']

22

### "Fancy" Indexing

Any boolean vector with the same shape as your `Index` can be used to select data.

In [12]:
temperature_series < 20

Monday       False
Tuesday      False
Wednesday     True
Thursday     False
Friday        True
Saturday     False
Sunday        True
dtype: bool

In [11]:
temperature_series[temperature_series < 20]

Wednesday    16
Friday       16
Sunday       15
dtype: int64

### Applying Functions

In [16]:
def f_to_c(temp_f):
    """ Convert temperature from degrees F to degrees C """
    temp_c = (5./9.) * (temp_f - 32)
    return temp_c

f_to_c(temperature_series)

Monday      -6.11
Tuesday     -3.89
Wednesday   -8.89
Thursday    -5.56
Friday      -8.89
Saturday    -6.11
Sunday      -9.44
dtype: float64

Automatic vectorization, like wrapping with `@numpy.vectorize`. 

### Applying Functions (cont'd)

In [17]:
np.mean(temperature_series)

19.428571428571427

In [20]:
(temperature_series.min(),
 temperature_series.quantile(0.33),
 temperature_series.mean(), 
 temperature_series.median(),
 temperature_series.quantile(0.66),
 temperature_series.max())

(15, 16.0, 19.428571428571427, 21.0, 21.0, 25)

## [DataFrame](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe)


<blockquote>
    <b>A two-dimensional, labeled data structure organized into rows and columns of potentially heterogeneous type</b>
</blockquote>


<img src="figs/dataframe.svg" style="width: 800px">

### Creating DataFrames

Constructing dictionaries of data is often the easiest way to create DataFrames

In [21]:
data = {
    'high_temp': [21, 35, 32, 40, 20],
    'low_temp': [14, 20, 29, 25, -1],
    'station': ['KEYW', 'KMDT', 'KGRI', 'KRNO', 'KTVC']
}
pd.DataFrame(data)

,high_temp,low_temp,station
0,21,14,KEYW
1,35,20,KMDT
2,32,29,KGRI
3,40,25,KRNO
4,20,-1,KTVC


In [22]:
pd.DataFrame(data, index=['Week {}'.format(i) for i in range(1, 6)])

,high_temp,low_temp,station
Week 1,21,14,KEYW
Week 2,35,20,KMDT
Week 3,32,29,KGRI
Week 4,40,25,KRNO
Week 5,20,-1,KTVC


**Nested dictionaries**

In [23]:
nest_data = {
   'high_temp': { 'Week 1': 21, 'Week 2': 35, 'Week 3': 32 },
   'low_temp': { 'Week 2': 20, 'Week 4': 25, 'Week 5': -1 },
   'station': { 'Week 1': 'KEYW', 'Week 3': 'KGRI', 'Week 4': 'KRNO'},
}
pd.DataFrame(nest_data)

,high_temp,low_temp,station
Week 1,21.0,NaN,KEYW
Week 2,35.0,20.0,NaN
Week 3,32.0,NaN,KGRI
Week 4,NaN,25.0,KRNO
Week 5,NaN,-1.0,NaN


**2D arrays**

In [24]:
mat = np.random.randint(-10, 20, size=(5, 3))
pd.DataFrame(mat, columns=['x', 'y', 'vel'], 
             index=pd.Index([1, 2, 3, 4, 5], name='sample'))

,x,y,vel
sample,,,
1,12,1,4
2,3,7,-9
3,-9,5,16
4,18,-9,4
5,13,16,4


**Structured / Record Arrays**

In [32]:
dt = [
    ('x', '<f8'), 
    ('y', '<f8'), 
    ('vel', '<f8')
]
mat_rec = np.rec.fromarrays(mat.T.tolist(), dtype=dt)
mat_rec['x'], mat_rec['y']

(array([ 12.,   3.,  -9.,  18.,  13.]), array([  1.,   7.,   5.,  -9.,  16.]))

In [33]:
pd.DataFrame(mat_rec, index=pd.Index(range(1, 6), name='sample_id'))

,x,y,vel
sample_id,,,
1,12.0,1.0,4.0
2,3.0,7.0,-9.0
3,-9.0,5.0,16.0
4,18.0,-9.0,4.0
5,13.0,16.0,4.0


In [35]:
df = pd.DataFrame(mat_rec, index=pd.Index(range(1, 6), name='sample_id'))

### DataFrame Structure

<center><img src="figs/dataframe.svg" style="width: 700px"></center>

In [36]:
df.index

RangeIndex(start=1, stop=6, step=1, name='sample_id')

In [37]:
df.columns

Index(['x', 'y', 'vel'], dtype='object')

In [38]:
df.values

array([[ 12.,   1.,   4.],
       [  3.,   7.,  -9.],
       [ -9.,   5.,  16.],
       [ 18.,  -9.,   4.],
       [ 13.,  16.,   4.]])

### DataFrame Structure

In [39]:
df.T

sample_id,1,2,3,4,5
x,12.0,3.0,-9.0,18.0,13.0
y,1.0,7.0,5.0,-9.0,16.0
vel,4.0,-9.0,16.0,4.0,4.0


In [40]:
df['x']

sample_id
1    12.0
2     3.0
3    -9.0
4    18.0
5    13.0
Name: x, dtype: float64

In [41]:
df.x

sample_id
1    12.0
2     3.0
3    -9.0
4    18.0
5    13.0
Name: x, dtype: float64

In [42]:
df.ix[1]

x      12.0
y       1.0
vel     4.0
Name: 1, dtype: float64

### Ingesting DataFrames from Disk

Go-to methods: **read_csv()**, **read_table()**, and **read_fmf()**

-   **filepath_or_buffer**: a path to a filename, or some other object
    with a `read()` method.
-   **sep**: delimiter separating values in your data. By default, this
    will be ',' for `read_csv()` and '\t' for `read_table()`, although
    you can pass it any regular expression. If you don't pass one,
    pandas will try to infer it automatically.
-   **delim_whitespace**: should pandas include arbitrary whitespace as a
    delimiter?
-   **header**: row numbers to use as column names, and where the data
    starts in the file
-   **names**: a list of column names to use; if the data has no header,
    this should be used with `header=None`
-   **index_col**: column to use as row labels
-   **skip{rows,footer}**: skip beginning/ending rows.


### Example: GISTEMP Global Monthly Mean Anomalies

In [47]:
!head -n 4 data/GLB.Ts+dSST.csv

Land-Ocean: Global Means
Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
1880,-.30,-.21,-.18,-.27,-.14,-.29,-.24,-.07,-.17,-.16,-.19,-.22,-.20,***,***,-.20,-.20,-.17
1881,-.10,-.14,.01,-.03,-.04,-.29,-.07,-.03,-.09,-.20,-.26,-.16,-.12,-.12,-.15,-.02,-.13,-.18


In [46]:
df = pd.read_csv("data/GLB.Ts+dSST.csv", 
                 skiprows=1, header=0, index_col=0, na_values='***')
df.head()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
Year,,,,,,,,,,,,,,,,,,
1880,-0.30,-0.21,-0.18,-0.27,-0.14,-0.29,-0.24,-0.07,-0.17,-0.16,-0.19,-0.22,-0.20,NaN,NaN,-0.20,-0.20,-0.17
1881,-0.10,-0.14,0.01,-0.03,-0.04,-0.29,-0.07,-0.03,-0.09,-0.20,-0.26,-0.16,-0.12,-0.12,-0.15,-0.02,-0.13,-0.18
1882,0.09,0.08,0.01,-0.20,-0.18,-0.25,-0.11,0.03,-0.01,-0.22,-0.21,-0.25,-0.10,-0.09,0.00,-0.12,-0.11,-0.15
1883,-0.34,-0.42,-0.18,-0.24,-0.25,-0.12,-0.08,-0.13,-0.18,-0.11,-0.20,-0.18,-0.20,-0.21,-0.34,-0.22,-0.11,-0.17
1884,-0.18,-0.12,-0.34,-0.36,-0.32,-0.38,-0.34,-0.26,-0.23,-0.22,-0.29,-0.29,-0.28,-0.27,-0.16,-0.34,-0.32,-0.25


### Example: GISTEMP Global Monthly Mean Anomalies

In [51]:
!head -n 9 data/GLB.Ts+dSST.txt

        GLOBAL Land-Ocean Temperature Index in 0.01 degrees Celsius   base period: 1951-1980

                    sources:  GHCN-v3 1880-11/2016 + SST: ERSST v4 1880-11/2016
                    using elimination of outliers and homogeneity adjustment
                    Notes: 1950 DJF = Dec 1949 - Feb 1950 ;  ***** = missing

                                                                     AnnMean
Year   Jan  Feb  Mar  Apr  May  Jun  Jul  Aug  Sep  Oct  Nov  Dec    J-D D-N    DJF  MAM  JJA  SON  Year
1880   -30  -21  -18  -27  -14  -29  -24   -7  -17  -16  -19  -22    -20 ***   ****  -20  -20  -17  1880


In [54]:
df = pd.read_table(
    'data/GLB.Ts+dSST.txt', delim_whitespace=True, index_col=0, skiprows=8, 
     names=['Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'July',
            'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], 
     header=None, usecols=range(13), engine='python', skipfooter=7, na_values='****')
df = df.drop('Year', axis=0).astype('float')
df.index = df.index.values.astype(np.int)
df.index.name = 'Year'
df.head()

,Jan,Feb,Mar,Apr,May,Jun,July,Aug,Sep,Oct,Nov,Dec
Year,,,,,,,,,,,,
1880,-30.0,-21.0,-18.0,-27.0,-14.0,-29.0,-24.0,-7.0,-17.0,-16.0,-19.0,-22.0
1881,-10.0,-14.0,1.0,-3.0,-4.0,-29.0,-7.0,-3.0,-9.0,-20.0,-26.0,-16.0
1882,9.0,8.0,1.0,-20.0,-18.0,-25.0,-11.0,3.0,-1.0,-22.0,-21.0,-25.0
1883,-34.0,-42.0,-18.0,-24.0,-25.0,-12.0,-8.0,-13.0,-18.0,-11.0,-20.0,-18.0
1884,-18.0,-12.0,-34.0,-36.0,-32.0,-38.0,-34.0,-26.0,-23.0,-22.0,-29.0,-29.0


## Indexing and Selection

Different ways to index into a DataFrame for different circumstances

**.ix[]** - NumPy-like label indexing on indices or labels (context-aware)

In [61]:
df.ix[:1900].head(3)

,Jan,Feb,Mar,Apr,May,Jun,July,Aug,Sep,Oct,Nov,Dec
Year,,,,,,,,,,,,
1880,-30.0,-21.0,-18.0,-27.0,-14.0,-29.0,-24.0,-7.0,-17.0,-16.0,-19.0,-22.0
1881,-10.0,-14.0,1.0,-3.0,-4.0,-29.0,-7.0,-3.0,-9.0,-20.0,-26.0,-16.0
1882,9.0,8.0,1.0,-20.0,-18.0,-25.0,-11.0,3.0,-1.0,-22.0,-21.0,-25.0


In [60]:
df.ix[2000:2010, ['Jun', 'July', 'Aug']].head(3)

,Jun,July,Aug
Year,,,
2000,43.0,41.0,43.0
2001,54.0,61.0,48.0
2002,54.0,62.0,55.0


In [59]:
df.ix[2000:2010, [5, 6, 7]].head(3)

,Jun,July,Aug
Year,,,
2000,43.0,41.0,43.0
2001,54.0,61.0,48.0
2002,54.0,62.0,55.0


**.iloc[]** - explicitly use 0-indexed values

In [62]:
df.iloc[20:30, [5, 6, 7]].head()

,Jun,July,Aug
Year,,,
1900,-15.0,-9.0,-4.0
1901,-10.0,-9.0,-13.0
1902,-35.0,-26.0,-28.0
1903,-45.0,-31.0,-44.0
1904,-50.0,-49.0,-44.0


**.loc[]** - explicitly use label-based indexing

In [63]:
df.loc[1900:1910, ['Jun', 'July', 'Aug']].head()

,Jun,July,Aug
Year,,,
1900,-15.0,-9.0,-4.0
1901,-10.0,-9.0,-13.0
1902,-35.0,-26.0,-28.0
1903,-45.0,-31.0,-44.0
1904,-50.0,-49.0,-44.0


**Boolean-based indexing**

In [66]:
df[df.Jun > df.July].head()

,Jan,Feb,Mar,Apr,May,Jun,July,Aug,Sep,Oct,Nov,Dec
Year,,,,,,,,,,,,
1890,-48.0,-48.0,-41.0,-37.0,-48.0,-27.0,-29.0,-36.0,-36.0,-22.0,-37.0,-30.0
1892,-26.0,-15.0,-36.0,-35.0,-25.0,-20.0,-28.0,-19.0,-25.0,-16.0,-50.0,-29.0
1898,-7.0,-34.0,-56.0,-33.0,-36.0,-21.0,-23.0,-23.0,-19.0,-32.0,-35.0,-22.0
1906,-31.0,-34.0,-15.0,-3.0,-21.0,-22.0,-27.0,-19.0,-25.0,-20.0,-39.0,-18.0
1912,-27.0,-13.0,-37.0,-20.0,-20.0,-26.0,-41.0,-52.0,-47.0,-56.0,-38.0,-42.0


or, with categoricals

In [79]:
df_cat = pd.melt(df.reset_index(), id_vars='Year', 
                 value_vars=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'July', 
                             'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], 
                 var_name='month',value_name='temperature')
df_cat[(df_cat.month.isin(['Jan', 'July', 'Dec'])) & (df_cat.Year == 1900)].head()

,Year,month,temperature
20,1900,Jan,-40.0
842,1900,July,-9.0
1527,1900,Dec,-14.0


### Hierarchical / Multi-Indexing

<img src="figs/multiindex.svg" style="width: 400px">

- Explicitly denote grouped entries, organized by "levels"
- Arises frequently in split-apply-combine and reshaping operations
- Easily select groups

In [80]:
df_cat['month'] = \
    df_cat['month'].astype("category", ordered=True,
                           categories=["Jan", "Feb", "Mar", "Apr", "May", "Jun",
                                       "July", "Aug", "Sep", "Oct", "Nov", "Dec"])
df_cat_mi = df_cat.set_index(['Year', 'month']).sortlevel()
df_cat_mi.head()

temperature
Year month             
1880 Jan          -30.0
     Feb          -21.0
     Mar          -18.0
     Apr          -27.0
     May          -14.0

In [82]:
df_cat_mi.loc[(slice(1900, 1905), slice('Jan', 'Mar')), :].head(5)

temperature
Year month             
1900 Jan          -40.0
     Feb           -8.0
     Mar            2.0
1901 Jan          -30.0
     Feb           -5.0

## Arithmetic

Like with `Series`, `DataFrame`s implement vectorized operations for most typical use cases.

**Example**: GISTEMP $\Delta$C anomalies to $\Delta$F 

In [83]:
res = (1./100.)*(9./5.) * df_cat_mi
res.head(3)

temperature
Year month             
1880 Jan          -0.54
     Feb          -0.38
     Mar          -0.32

Wrapped in a function...

In [84]:
def calc_f_anom(c_anom):
    return c_anom*(1./100.)*(9./5.)

calc_f_anom(df_cat_mi).head(3)

temperature
Year month             
1880 Jan          -0.54
     Feb          -0.38
     Mar          -0.32

Using NumPy ufuncs...

In [88]:
res = df_cat_mi - np.mean(df_cat_mi)
res.head(3)

temperature
Year month             
1880 Jan         -32.31
     Feb         -23.31
     Mar         -20.31

### Index Alignment

The most critical feature of pandas is its efficient, performant alignment of indices across data structures.

In [93]:
x = pd.Series(np.random.randint(0, 10, 6), index=range(6))
y = pd.Series(np.random.randint(0, 10, 6), index=range(3, 9))

df = pd.DataFrame({'x': x, 'y': y})
df

,x,y
0,6.0,NaN
1,9.0,NaN
2,5.0,NaN
3,4.0,1.0
4,7.0,9.0
5,7.0,9.0
6,NaN,5.0
7,NaN,0.0
8,NaN,6.0


In [94]:
x + y

0     NaN
1     NaN
2     NaN
3     5.0
4    16.0
5    16.0
6     NaN
7     NaN
8     NaN
dtype: float64

Removing padded indices after alignment

In [95]:
(x + y).dropna()

3     5.0
4    16.0
5    16.0
dtype: float64

### Function Mapping

General syntax for column/row-wise or element-wise function vectorization

**apply()** - column/row-wise vectorization

In [96]:
def data_range(x):
    return x.max() - x.min()

df.apply(data_range)

x    5.0
y    9.0
dtype: float64

In [ ]:
def range_stats(x):
    return pd.Series([x.min(), x.median(), x.max()], 
                     index=['min', 'median', 'max'])

df.apply(range_stats)

... automatic re-indexing on new labels

### Descriptive Statistics

Many standard statistical calculations are built-in to pandas, making exploratory data analysis very easy.

In [97]:
n = 10
df = pd.DataFrame({'x': np.random.randn(n), 'y': np.random.randn(n)},
                  index=range(1, n+1))
df.describe()

,x,y
count,1.00e+01,10.00
mean,-1.65e-01,-0.62
std,6.86e-01,0.63
min,-1.10e+00,-1.29
25%,-7.21e-01,-1.19
50%,-7.07e-03,-0.75
75%,2.57e-01,-0.08
max,1.06e+00,0.32


In [99]:
def desc_stats(x):
    return pd.Series([x.mean(), x.mad(), x.quantile(.25), x.var(), 
                      x.max(), x.argmax(), x.idxmax()],
                     index=['mean', 'mad', '25%', 'var', 'max', 'argmax', 'idxmax'])
df.apply(desc_stats)

,x,y
mean,-0.16,-0.62
mad,0.56,0.56
25%,-0.72,-1.19
var,0.47,0.39
max,1.06,0.32
argmax,1.00,4.00
idxmax,1.00,4.00


## Timeseries

### Time in Python

Python already ships with libraries for manipulating date and timestamps

In [109]:
from datetime import datetime

now = datetime.now()
print(now)

2017-01-19 01:47:14.988048


In [101]:
now.year, now.month, now.day, now.hour

(2017, 1, 19, 1)

**Time deltas** - why keep track of a complicated base-60 number system when something else can do it for you?

In [106]:
delta = datetime(2017, 1, 18) - datetime(2017, 1, 17, 6, 43)
delta.total_seconds()

62220.0

In [108]:
from datetime import timedelta

delta = timedelta(hours=3, minutes=30)
print(delta + datetime(2017, 1, 18, 12, 15))

2017-01-18 15:45:00


**Parsing / Shuffling Timestamps**

In [110]:
now.strftime("%A, %b %d at %I:%m%p")

'Thursday, Jan 19 at 01:01AM'

In [112]:
timestamp = "2017-01-18"
x = datetime.strptime(timestamp, "%Y-%m-%d") - timedelta(hours=3, minutes=3)
print(x)

2017-01-17 20:57:00


Python adheres to [C-standard time format codes](https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior)

### Time Indices

Using date or timestamps as an **Index** automatically promotes your **Series** to a **TimeSeries** object with some additional useful features.

In [113]:
times = [datetime(2017, 1, day) for day in range(1, 31)]
ts = pd.Series(np.random.randint(0, 10, len(times)), index=times)
ts.head()

2017-01-01    2
2017-01-02    1
2017-01-03    3
2017-01-04    9
2017-01-05    8
dtype: int64

In [114]:
ts.index[:5]

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05'],
              dtype='datetime64[ns]', freq=None)

In [115]:
ts.index[5]

Timestamp('2017-01-06 00:00:00')

**`pd.date_range()`** - for quickly creating time indices

Option 1) Frequency between given start/end datetimes

In [116]:
pd.date_range(start='1/1/2017', end='1/30/2017', freq='3D')

DatetimeIndex(['2017-01-01', '2017-01-04', '2017-01-07', '2017-01-10',
               '2017-01-13', '2017-01-16', '2017-01-19', '2017-01-22',
               '2017-01-25', '2017-01-28'],
              dtype='datetime64[ns]', freq='3D')

Option 2) Begin date with specified number of periods

In [117]:
pd.date_range(start='1/1/2017', freq='2W', periods=6)

DatetimeIndex(['2017-01-01', '2017-01-15', '2017-01-29', '2017-02-12',
               '2017-02-26', '2017-03-12'],
              dtype='datetime64[ns]', freq='2W-SUN')

### Time Selection

Because time has a natural ordering, it's easy to slice the data you want out of a **TimeSeries**

In [118]:
times = pd.date_range(start='2017-01-01', end='2017-01-031', freq='1H')
n = len(times)
weather_data = pd.DataFrame({'temperature': np.random.randint(-10, 54, n),
                             'wind_speed': np.abs(np.random.randn(n)),
                             'wind_dir': np.random.randint(0, 360, n)},
                            index=times)
weather_data.head()

,temperature,wind_dir,wind_speed
2017-01-01 00:00:00,47,163,0.46
2017-01-01 01:00:00,27,269,1.39
2017-01-01 02:00:00,48,5,0.20
2017-01-01 03:00:00,-7,284,0.21
2017-01-01 04:00:00,22,61,1.21


Select a single day:

In [121]:
weather_data.loc["1/13/2017"].head(3)

,temperature,wind_dir,wind_speed
2017-01-13 00:00:00,51,288,1.66
2017-01-13 01:00:00,25,112,0.09
2017-01-13 02:00:00,30,145,0.26


Select an arbitrary 12-hour period:

In [122]:
weather_data.loc["1/28/2017 21:00:00":"1/29/2017 03:00:00"].head(3)

,temperature,wind_dir,wind_speed
2017-01-28 21:00:00,43,237,1.21
2017-01-28 22:00:00,29,142,0.24
2017-01-28 23:00:00,38,25,0.59


Alternative way to window data:

In [124]:
weather_data.truncate(before="1/28/2017 12:00:00", after="1/31/2017").head(3)

,temperature,wind_dir,wind_speed
2017-01-28 12:00:00,42,35,0.61
2017-01-28 13:00:00,-6,285,0.38
2017-01-28 14:00:00,22,33,1.66


### Resampling

- **Downsampling** or "aggregation": high frequency -> low frequency
- **Upsampling** or "interpolation": low frequency -> high frequency

In [125]:
daily = weather_data.resample('D').max()
daily.head()

,temperature,wind_dir,wind_speed
2017-01-01,53,307,2.14
2017-01-02,49,358,3.07
2017-01-03,53,329,2.12
2017-01-04,51,349,2.58
2017-01-05,52,347,2.05


In [127]:
weekly = weather_data.loc['2017-01-01':'2017-01-10'].resample('W-WED').mean()
weekly.head()

,temperature,wind_dir,wind_speed
2017-01-04,24.06,173.56,0.89
2017-01-11,20.97,169.33,0.74


---

## Digging deeper...

The basics are important to know, but pandas' utility goes far beyond simple data structures.


## [US Historical Climatology Network](http://cdiac.ornl.gov/epubs/ndp/ushcn/ushcn.html)


<img src="figs/ushcn_stations.png">

- Daily / Monthly temperature and precipitation statistics from 1218 sites across CONUS
- Some stations have over 100 years of data
- Data is quality-controlled and "homogenized"


In [128]:
df = pd.read_pickle("data/ushcn2014_tob_tmax.p")
stations = pd.read_pickle("data/ushcn-stations.p")

In [129]:
df.head()

,STATION,YEAR,TMAX,DM,QC,DS,month,date,COOP_ID
0,USH00011084,1891,NaN,,,,1,1891-01-01,11084
1,USH00011084,1892,532.0,,,,1,1892-01-01,11084
2,USH00011084,1893,552.0,,,,1,1893-01-01,11084
3,USH00011084,1894,656.0,,,,1,1894-01-01,11084
4,USH00011084,1895,584.0,,,,1,1895-01-01,11084


In [130]:
stations.head()

,LAT,LON,ELEV,STATE,NAME,COMP1,COMP2,COMP3,dUTC
COOP_ID,,,,,,,,,
11084,31.06,-87.05,25.9,AL,BREWTON 3 SSE,------,------,------,6
12813,30.55,-87.88,7.0,AL,FAIRHOPE 2 NE,------,------,------,6
13160,32.83,-88.13,38.1,AL,GAINESVILLE LOCK,011694,------,------,6
13511,32.70,-87.58,67.1,AL,GREENSBORO,------,------,------,6
13816,31.87,-86.25,132.0,AL,HIGHLAND HOME,------,------,------,6


### Merging datasets

In [132]:
merged = df.join(stations, on='COOP_ID')
merged.head()

,STATION,YEAR,TMAX,DM,QC,DS,month,date,COOP_ID,LAT,LON,ELEV,STATE,NAME,COMP1,COMP2,COMP3,dUTC
0,USH00011084,1891,NaN,,,,1,1891-01-01,11084,31.06,-87.05,25.9,AL,BREWTON 3 SSE,------,------,------,6
1,USH00011084,1892,532.0,,,,1,1892-01-01,11084,31.06,-87.05,25.9,AL,BREWTON 3 SSE,------,------,------,6
2,USH00011084,1893,552.0,,,,1,1893-01-01,11084,31.06,-87.05,25.9,AL,BREWTON 3 SSE,------,------,------,6
3,USH00011084,1894,656.0,,,,1,1894-01-01,11084,31.06,-87.05,25.9,AL,BREWTON 3 SSE,------,------,------,6
4,USH00011084,1895,584.0,,,,1,1895-01-01,11084,31.06,-87.05,25.9,AL,BREWTON 3 SSE,------,------,------,6
